In [1]:
%reload_ext watermark
%watermark -uniz --author "Prayson W. Daniel" -vm -p polars,dspy

Author: Prayson W. Daniel

Last updated: 2024-12-17T13:15:19.974441+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.29.0

polars: 1.7.1
dspy  : 2.5.43

Compiler    : Clang 15.0.0 (clang-1500.3.9.4)
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit



In [2]:
from typing import Final, Literal

import dspy
import polars as pl

In [3]:
MODEL: Final = "gemma2"

VALID_EMOTIONS: tuple[str, ...] = tuple(
    {
        "admiration",
        "approval",
        "gratitude",
        "optimism",
        "joy",
        "neutral",
        "disapproval",
        "disappointment",
        "love",
        "relief",
        "excitement",
        "desire",
        "caring",
        "pride",
        "realization",
        "disgust",
        "annoyance",
        "remorse",
        "amusement",
        "anger",
        "grief",
        "curiosity",
        "fear",
        "surprise",
        "sadness",
        "embarrassment",
        "confusion",
        "nervousness",
    }
)

class Emotion(dspy.Signature):
    """Classify emotions of a given sentence."""
    sentence: str = dspy.InputField()
    kind: Literal[VALID_EMOTIONS] = dspy.OutputField()
    reason: str = dspy.OutputField(
        description="A short third person reason for selected the emotion"
    )

In [4]:
dspy.configure(lm=dspy.LM("ollama_chat/gemma2", 
                          api_base="http://localhost:11434", 
                          api_key="NotNeeded",
                          temperature=0.0,)
              )

In [5]:
classifier = dspy.Predict(Emotion)

In [6]:
pl.Config.set_fmt_str_lengths(100).set_tbl_rows(
    -1
)  # show more text. default is 25, show all rowss

dataf = pl.read_json("../data/examples.json").explode(["emotion", "text"])

In [7]:
dataf.sample(3, seed=7)

emotion,text
str,str
"""confusion""","""The situation is so unclear right now, it’s leaving me confused."""
"""love""","""My love for nature grows with every hike in the mountains."""
"""caring""","""It’s important to care for your mental health as much as your physical health."""


In [8]:
classifier(sentence="I dont' know how to think about where the world is going?")

Prediction(
    kind='confusion',
    reason='The sentence expresses uncertainty about the future.'
)

In [9]:
# python function failed TypeError: Predict.__call__() takes 1 positional argument but 2 were given
def classifierx(sentence: str) -> dspy.Prediction:
    return classifier(sentence=sentence)

In [10]:
(
    dataf.sample(8, seed=42)
    .with_columns(
        pl.col("text")
        .map_elements(classifierx, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.kind, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reason, return_dtype=pl.String)
        .alias("reason"),
    )
)

emotion,text,prediction,reason
str,str,str,str
"""confusion""","""His mixed signals have left me feeling utterly confused.""","""confusion""","""The sentence explicitly states feeling confused due to mixed signals."""
"""joy""","""I can’t stop smiling after watching that movie. Such a feel-good story!""","""joy""","""The speaker expresses happiness and positivity about the movie."""
"""disappointment""","""I’m disappointed that the event was canceled last minute.""","""disappointment""","""The speaker expresses sadness about the event's cancellation."""
"""anger""","""I’m furious that the package didn’t arrive on time.""","""anger""","""The speaker is expressing strong negative feelings about the delayed package."""
"""realization""","""It’s in moments like these that you realize what really matters in life.""","""realization""","""The sentence suggests a moment of insight and understanding about life's priorities."""
"""love""","""My love for nature grows with every hike in the mountains.""","""love""","""The speaker expresses a deep affection for nature."""
"""caring""","""She’s always caring for others, making sure everyone feels valued.""","""pride""","""She is being praised for her kindness and thoughtfulness."""
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!""","""admiration""","""The speaker is impressed by his accomplishment."""


In [11]:
classifier.signature

Emotion(sentence -> kind, reason
    instructions='Classify emotions of a given sentence.'
    sentence = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Sentence:', 'desc': '${sentence}'})
    kind = Field(annotation=Literal['disapproval', 'nervousness', 'neutral', 'approval', 'pride', 'amusement', 'remorse', 'gratitude', 'desire', 'annoyance', 'sadness', 'caring', 'embarrassment', 'optimism', 'joy', 'love', 'grief', 'relief', 'disappointment', 'disgust', 'confusion', 'fear', 'anger', 'surprise', 'admiration', 'excitement', 'realization', 'curiosity'] required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Kind:', 'desc': '${kind}'})
    reason = Field(annotation=str required=True description='A short third person reason for selected the emotion' json_schema_extra={'__dspy_field_type': 'output', 'desc': 'A short third person reason for selected the emotion', 'prefix': 'Reason:'})
)

## Opens a New Possibilities

In [12]:
thinker = dspy.ChainOfThought(Emotion)

In [13]:
thinker(sentence="I dont' know how to think about where the world is going?")

In [16]:
# ollama does not allow, yet, multiple loops: Useful in RAG (context)
try:
    thinkers = dspy.ChainOfThought(Emotion, n=3)
    thinkers(sentence="I dont' know how to think about where the world is going?")
except Exception as e:
    print(e)

litellm.UnsupportedParamsError: ollama_chat does not support parameters: {'n': 3}, for model=gemma2. To drop these, set `litellm.drop_params=True` or for proxy:

`litellm_settings:
 drop_params: true`



---
Given that DSPy is modular, we don't have to wait for Ollama support

---

In [ ]:
class Emotions(dspy.Signature):
    """Judge the emotions of a given sentence given context"""
    sentence: str = dspy.InputField()
    emotion: Literal[VALID_EMOTIONS] = dspy.OutputField()
    context: list[Emotion] = dspy.OutputField(
        description="Prediction and reasons from experts"
    )

In [18]:

class Thinkers(dspy.Module):
    def __init__(self, n: int = 3):
        self.n = n # loops
        self.thinker = dspy.ChainOfThought(Emotion)
        self.judge = dspy.ChainOfThought(Emotions)

    def forward(self, sentence:str):
        context = []
        for _ in range(self.n):
            response = self.thinker(sentence=sentence)
            context += [response]
        
        
        result = self.judge(context=context, sentence=sentence)
        return result
        

In [19]:
thinkers = Thinkers(n=2)

In [20]:
thinkers(sentence="I dont' know how to think about where the world is going?")

Prediction(
    reasoning='The sentence expresses uncertainty and worry about the future direction of the world. This suggests a feeling of unease and potential anxiety.',
    emotion='nervousness',
    context=[Emotion(sentence="I dont' know how to think about where the world is going?", kind='nervousness', reason='The sentence expresses uncertainty and worry about the future, indicating a feeling of unease.')]
)

In [21]:
def classifier(sentence:str) -> Emotions:
    result = thinkers(sentence=sentence)
    return result

In [22]:
(
    dataf.sample(8, seed=42)
    .with_columns(
        pl.col("text")
        .map_elements(classifier, return_dtype=pl.Object)
        .alias("prediction")
    )
    .with_columns(
        pl.col("prediction")
        .map_elements(lambda p: p.emotion, return_dtype=pl.String)
        .alias("prediction"),
        pl.col("prediction")
        .map_elements(lambda p: p.reasoning, return_dtype=pl.String)
        .alias("reasoning"),
    )
)

emotion,text,prediction,reasoning
str,str,str,str
"""confusion""","""His mixed signals have left me feeling utterly confused.""","""confusion""","""The sentence expresses a feeling of uncertainty and perplexity due to conflicting or unclear message…"
"""joy""","""I can’t stop smiling after watching that movie. Such a feel-good story!""","""joy""","""The sentence expresses a positive feeling of happiness and satisfaction after watching a movie. The …"
"""disappointment""","""I’m disappointed that the event was canceled last minute.""","""disappointment""","""The sentence expresses regret and sadness about an event being canceled unexpectedly."""
"""anger""","""I’m furious that the package didn’t arrive on time.""","""anger""","""The sentence expresses strong negative feelings due to the package's delayed arrival. The word ""furi…"
"""realization""","""It’s in moments like these that you realize what really matters in life.""","""realization""","""The sentence suggests a moment of introspection and reflection, leading to an understanding of life'…"
"""love""","""My love for nature grows with every hike in the mountains.""","""joy""","""The sentence expresses a positive sentiment about nature and hiking. The speaker's love for nature i…"
"""caring""","""She’s always caring for others, making sure everyone feels valued.""","""caring""","""The sentence describes someone consistently showing kindness and consideration towards others. This …"
"""admiration""","""He ran a marathon in record time despite his injury. What a remarkable feat!""","""admiration""","""The sentence expresses admiration for someone's accomplishment despite facing adversity."""
